In [1]:
import pandas as pd
import ast
import numpy as np
import cv2
from skimage.morphology import skeletonize
from shapely.geometry import LineString, Polygon

def to_tuple(t):
    return ast.literal_eval(t)

def to_float(a):
    return np.array(a[1:-1].split(',')).astype(float)

def maxDeviationThresh(hist):
    maximum = max(hist)
    index_max = list(hist).index(maximum)
    index_min = 0
    for i in range(0, index_max):
        if not hist[i] and hist[i + 1]:
            index_min = i
            break

    distances = []
    x1 = index_min
    y1 = hist[index_min]
    x2 = index_max
    y2 = hist[index_max]
    for i in range(index_min + 1, index_max):
        x0 = i
        y0 = hist[i]
        distance = np.abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1) / np.sqrt(
            (y2 - y1) ** 2 + (x2 - x1) ** 2)
        distances.append(distance)
    if index_min < index_max - 1:
        T_index = distances.index(max(distances))
    else:
        T_index = -index_min
    return T_index + index_min


def extract_drawing(image):
    dst = cv2.bilateralFilter(image, 10, sigmaColor=15, sigmaSpace=15)
    # dst = img.copy()
    # max_occ = np.bincount(dst[dst > 0]).argmax()
    # dst[dst == 0] = max_occ
    threshed = np.ones(dst.shape, np.uint8) * 255
    if np.any(dst < 255):
        hist, _ = np.histogram(dst[dst < 255].flatten(), range(257))
        thresh_val = maxDeviationThresh(hist)
        #print(thresh_val)
        mask = dst < thresh_val
        threshed[mask] = 0
    return threshed


def getBackground(external, img, morph=True, ret_hier=False, internal=None):
    background = np.zeros_like(img)
    points = np.array([external]).reshape((4, 1, 2))
    background = cv2.fillConvexPoly(background, points, (255, 255, 255))
    background = cv2.bitwise_and(img, background)
    if internal is not None:
      int_points = np.array([internal]).reshape((4, 1, 2))
      background = cv2.fillConvexPoly(background, int_points, (255, 255, 255))
    '''overlap = cv2.polylines(cv2.cvtColor(img.copy(), cv2.COLOR_GRAY2RGB), [points], True, (255, 0, 0), 1)
    if internal is not None:
      overlap = cv2.polylines(overlap, [int_points], True, (255, 0, 0), 1)
      plt.imshow(overlap)
      plt.show()'''
    background[background == 0] = 255
    background = extract_drawing(background)
    if morph:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
        # background = cv2.bitwise_not(background)
        background = cv2.bitwise_not(cv2.erode(background, kernel))
        background = skeletonize(background / 255, method='lee').astype(np.uint8)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        background = cv2.dilate(background, kernel)
    else:
        background = cv2.bitwise_not(background)
        background = skeletonize(background / 255, method='lee').astype(np.uint8)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        background = cv2.dilate(background, kernel)
    if internal is not None:    
      plt.imshow(background, cmap='gray')
      plt.show()
    cnts, hier = cv2.findContours(background, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if ret_hier:
        return background, cnts, hier
    else:
        return background, cnts

In [2]:
class Pattern10:
  def __init__(self, img, drawing, model_diag, scaler_diag, m, s, img_path):
    self.img = img    
    self.drawing = drawing
    self.model_diag = model_diag
    self.scaler_diag = scaler_diag
    self.m = m
    self.s = s
    self.img_path = img_path
  
  def get_score(self, diag, oriz):
    coords = [742, 287, 829, 373]
    filepath = '../results/scores.csv'    
    df_rail = pd.read_csv(filepath, header=0, usecols=['names', 'scores', 'rect'], index_col='names', converters={'scores': to_float, 'rect': to_tuple})
    if self.img_path[:-4] in df_rail.index:
      rail_bbox = df_rail.loc[self.img_path[:-4], 'rect'][1]
      external = [(rail_bbox[0], rail_bbox[1]), (rail_bbox[0]+rail_bbox[2], rail_bbox[1]), (rail_bbox[0]+rail_bbox[2], rail_bbox[1]+rail_bbox[3]), (rail_bbox[0], rail_bbox[1]+rail_bbox[3])]
      background_rail, _ = getBackground(external, self.img, False)
      pixel_rail = np.sum(np.divide(background_rail, 255))
      rail_prediction = self.model_diag.predict(self.scaler_diag.transform(np.array([pixel_rail]).reshape(-1,1)))
      score_rail = self.s.transform(df_rail.loc[self.img_path[:-4], 'scores'][1].reshape(-1,1))   
      rail_score = self.m.predict(score_rail)
      
      if rail_score == 1:
        self.drawing = cv2.rectangle(self.drawing, (rail_bbox[0], rail_bbox[1]),
                                        (rail_bbox[0] + rail_bbox[2], rail_bbox[1] + rail_bbox[3]), (255, 0, 0), 2)
        bbox_fig = Polygon(external)
        if diag is not None:
          diag_fig = LineString(diag)
        else:
          diag_fig = LineString([[852, 219],[382, 537]])
        splitted = diag_fig.intersection(bbox_fig)
        if not splitted.is_empty:
          split_coord = splitted.xy  
          int_points = np.array(list(zip(split_coord[0], split_coord[1]))).astype(int)
        p1 = splitted.is_empty or max(int_points[:, 0]) < rail_bbox[0] + 2*rail_bbox[2]/3 and max(int_points[:,1]) < rail_bbox[1] + 2*rail_bbox[3]/3 
        if not p1:
          print('PATTERN10: diagonale mal posizionata')
        if oriz is not None:
          oriz_fig = LineString(oriz)
        else:
          dist = int((537 - 219) / 2)
          oriz_fig = LineString([[382, 537 - dist], [852, 537 - dist]])
        splitted = oriz_fig.intersection(bbox_fig)
        if not splitted.is_empty:
            split_coord = splitted.xy
            int_points = np.array(list(zip(split_coord[0], split_coord[1]))).astype(int)
        p2 = splitted.is_empty or min(int_points[:, 1]) > rail_bbox[1] + 2*rail_bbox[3]/3 
        if not p2:
            print('PATTERN10: orizzontale mal posizionata')
        if p1 and p2:
          label_rail = 3
        else:
            label_rail = 2
      
      else:
        if rail_prediction == 1:
          self.drawing = cv2.rectangle(self.drawing, (rail_bbox[0], rail_bbox[1]), (rail_bbox[0]+rail_bbox[2], rail_bbox[1]+rail_bbox[3]), (0,0,255), 2)
          print('PATTERN10: disegno impreciso')
          label_rail = 1
        else:
          self.drawing = cv2.rectangle(self.drawing, (rail_bbox[0], rail_bbox[1]), (rail_bbox[0]+rail_bbox[2], rail_bbox[1]+rail_bbox[3]), (0,0,255), 2)
          print('PATTERN10: disegno assente')
          label_rail = 0
    else:
      x = coords[0] 
      y = coords[1] 
      w = np.abs(coords[0] - coords[2])
      h = np.abs(coords[1] - coords[3])
      external = [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]
      background_rail, _ = getBackground(external, self.img, False)
      pixel_rail = np.sum(np.divide(background_rail, 255))
      rail_prediction = self.model_diag.predict(self.scaler_diag.transform(np.array([pixel_rail]).reshape(-1,1)))
      if rail_prediction == 1:
        label_rail = 1
      else:
        label_rail = 0
    return self.drawing, label_rail